In [0]:
data = [
 ("C001", "2024-01-01"),
 ("C001", "2024-01-04"),
 ("C001", "2024-01-06"),
 ("C002", "2024-01-03"),
 ("C002", "2024-01-05"),
]

df = spark.createDataFrame(data, ["customer_id", "billing_date"])

df.display()

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, datediff, date_add, date_sub

In [0]:
customer_window=Window.partitionBy("customer_id").orderBy("billing_date")

prev_date_df=df.withColumn("prev_date",lag("billing_date",1).over(customer_window))

prev_date_df.display()

gap_df=prev_date_df.filter(datediff(col("billing_date"),col("prev_date"))>1)

gap_df.display()

result_df=gap_df.withColumn("missing_from",date_add(col("prev_date"),1)).withColumn("missing_to",date_sub(col('billing_date'),1))

result_df.display()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min, max, explode, sequence, to_date

# Initialize Spark
spark = SparkSession.builder.appName("MissingBillingDates").getOrCreate()

# Sample data
data = [
    ("C001", "2024-01-01"),
    ("C001", "2024-01-02"),
    ("C001", "2024-01-04"),
    ("C001", "2024-01-06"),
    ("C002", "2024-01-03"),
    ("C002", "2024-01-05"),
]

# Create DataFrame
df = spark.createDataFrame(data, ["customer_id", "billing_date"]) \
    .withColumn("billing_date", to_date("billing_date"))
df.display()
# Step 1: Get min and max billing date per customer
date_range_df = df.groupBy("customer_id").agg(
    min("billing_date").alias("start_date"),
    max("billing_date").alias("end_date")
)
date_range_df.display()
# Step 2: Generate complete date range per customer
full_dates_df = date_range_df.withColumn("billing_date", explode(
    sequence(col("start_date"), col("end_date"))
)).select("customer_id", "billing_date")
full_dates_df.display()
# Step 3: Find missing dates via anti-join
missing_dates_df = full_dates_df.join(df, on=["customer_id", "billing_date"], how="anti")
# Show results
missing_dates_df.orderBy("customer_id", "billing_date").display()

In [0]:
#input: data = [(1,), (2,), (4,), (6,), (8,), (9,)]

#output expected : [(3,), (5,), (7,)]

#spark = SparkSession.builder.getOrCreate()

# Actual numbers DataFrame
data = [(1,), (2,), (4,), (6,), (8,), (9,)]
numbers_df = spark.createDataFrame(data, ["num"])

#Tip1 > create dataframe having values 1 to 9

# Full range DataFrame (1 to 9)
full_range_df = spark.range(1, 10).toDF("num")

# Use anti join to find missing numbers
missing_numbers = full_range_df.join(numbers_df, on="num", how="left_anti")
missing_numbers.show()

In [0]:
from pyspark.sql.window import *
from pyspark.sql.functions import *

data = [
    (101,["P1","P2"]),
    (102,["P1"]),
    (103,["P2","P3"])
]

df = spark.createDataFrame(data,["id","projects"])
df.display()

df_splitedlist = df.withColumn("projects", explode("projects"))
df_splitedlist.display()

df_result = df_splitedlist.groupBy("projects").agg(countDistinct("id").alias("count"))
df_result.display()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag, lead
from pyspark.sql.window import Window

# Setup
spark = SparkSession.builder.getOrCreate()

# Sample sales data
data = [
    ("ProductA", "2024-01", 100),
    ("ProductA", "2024-02", 120),
    ("ProductA", "2024-03", 90),
    ("ProductB", "2024-01", 200),
    ("ProductB", "2024-02", 210),
    ("ProductB", "2024-03", 200),
]

columns = ["Product", "Month", "Revenue"]
df = spark.createDataFrame(data, columns)

In [0]:
# Define window partitioned by product and ordered by month
windowSpec = Window.partitionBy("Product").orderBy("Month")

# Add lag and lead columns
df_with_lag_lead = df.withColumn("Prev_Revenue", lag("Revenue", 1).over(windowSpec)) \
                     .withColumn("Next_Revenue", lead("Revenue", 1).over(windowSpec))

df_with_lag_lead.display()

In [0]:
from pyspark.sql.functions import col, lit, concat

df_with_lag_lead_wri = df_with_lag_lead.withColumn(
    "Revenue_Gap", 
    concat(
        (((col("Revenue") - col("Prev_Revenue")) / col("Revenue")) * lit(100)).cast("string"), 
        lit('%')
    )
).withColumn('Flag', lit('Flag'))
df_with_lag_lead_wri.display()

In [0]:
from pyspark.sql.functions import col, lit, concat

df_with_lag_lead_wri = df_with_lag_lead.withColumn(
    "Revenue_Gap", 
    concat(
        (((col("Revenue") - col("Prev_Revenue")) / col("Revenue")) * lit(100)).cast("string"), 
        lit('%')
    )
).withColumn('Flag1', lit('Flag'))
df_with_lag_lead_wri.display()

In [0]:
df_with_lag_lead_wri.write.mode("append").option("mergeSchema", "true").saveAsTable("rro.sales_data")

In [0]:
%sql
select * from rro.sales_data version as of 3;

In [0]:
%sql
RESTORE TABLE rro.sales_data TO VERSION AS OF 1;

In [0]:
_sqldf.display()

In [0]:
%sql
select * from rro.sales_data

In [0]:
from pyspark.sql.functions import col, lit, concat

df_with_lag_lead_wri = df_with_lag_lead.withColumn(
    "Revenue_Gap", 
    concat(
        (((col("Revenue") - col("Prev_Revenue")) / col("Revenue")) * lit(100)).cast("string"), 
        lit('%')
    )
).withColumn('Flag', lit('Flag'))
df_with_lag_lead_wri.display()

In [0]:
required_columns = ["Product", "Month", "Revenue", "Prev_Revenue", "Next_Revenue", "Revenue_Gap", "Flag", "Flag1"]
available_columns = df_with_lag_lead_wri.columns
print(available_columns)
valid_columns = [col for col in required_columns if col in available_columns]
print(valid_columns)
df_with_lag_lead_wri.select(*valid_columns).write.mode("overwrite").saveAsTable("rro.sales_data")

In [0]:
%sql
select * from rro.sales_data

In [0]:
try:
    df_with_lag_lead_wri.select("Product","Month","Revenue","Prev_Revenue","Next_Revenue","Revenue_Gap","Flag","Flag1").write.mode("append").saveAsTable("rro.sales_data")
except Exception as e:
  print("An error ocured")

In [0]:
%sql
select * from rro.sales_data

In [0]:
from pyspark.sql import DataFrame

def validate_schema(df: DataFrame, expected_columns: list, strict: bool = False) -> bool:
    """
    Validates that all expected_columns exist in the DataFrame.
    
    Parameters:
    - df: Spark DataFrame to check
    - expected_columns: List of required column names
    - strict: If True, also checks that no extra columns exist
    
    Returns:
    - True if schema matches; False otherwise
    """
    actual_columns = set(df.columns)
    expected_set = set(expected_columns)
    
    missing = expected_set - actual_columns
    extras = actual_columns - expected_set if strict else set()
    
    if missing:
        print(f"[❌] Missing columns: {sorted(missing)}")
    if extras:
        print(f"[⚠️] Unexpected columns (strict mode): {sorted(extras)}")
    
    return not missing and not extras

In [0]:
expected_cols = ["Product", "Month", "Revenue", "Prev_Revenue", 
                 "Next_Revenue", "Revenue_Gap", "Flag", "Flag1"]

is_valid = validate_schema(df_with_lag_lead_wri, expected_cols, strict=False)

if is_valid:
    df_with_lag_lead_wri.select(*expected_cols).write.mode("append").saveAsTable("rro_sales_data")
else:
    print("[🚫] Schema mismatch. Fix required before write operation.")

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Fetch schema from rro_sales_data
table_schema = spark.table("rro.sales_data").schema
display(table_schema)
# Extract column names
expected_cols = [field.name for field in table_schema]
display(expected_cols)

In [0]:
is_valid = validate_schema(df_with_lag_lead_wri, expected_cols, strict=False)

if is_valid:
    df_with_lag_lead_wri.select(*expected_cols).write.mode("append").saveAsTable("rro_sales_data")
else:
    print("[❌] Schema mismatch. Fix required before write operation.")

In [0]:
from pyspark.sql.functions import col, mean

# Sample DataFrame with missing values
data = [(1, None), (2, 3.0), (None, 4.0), (4, 5.0)]
columns = ["col1", "col2"]
df = spark.createDataFrame(data, columns)

# Calculate column-wise means (excluding nulls)
means = df.select([mean(col(c)).alias(c) for c in df.columns]).collect()[0].asDict()

# Fill nulls with computed means
df_filled = df.fillna(means)

df_filled.show()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast


# Create Spark session
spark = SparkSession.builder.appName("MultiColumnJoin").getOrCreate()

# Sample DataFrames
data1 = [(1, "A", 100), (2, "B", 200), (3, "C", 300)]
data2 = [(1, "A", "X"), (2, "B", "Y"), (4, "D", "Z")]

df1 = spark.createDataFrame(data1, ["id", "code", "value"])
df2 = spark.createDataFrame(data2, ["id", "code", "flag"])

# Perform inner join on multiple columns: 'id' and 'code'
joined_df1 = df1.join(df2, on=["id", "code"], how="inner")
joined_df2 = df1.join(broadcast(df2), (df1.id==df2.id) & (df1.code==df2.code), how="inner")

# Show result
joined_df1.show()
joined_df2.show()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, avg

# Start Spark session
spark = SparkSession.builder.appName("MovingAverage").getOrCreate()

# Sample data (e.g., daily sales)
data = [
    ("2023-01-01", 100),
    ("2023-01-02", 150),
    ("2023-01-03", 200),
    ("2023-01-04", 250),
    ("2023-01-05", 300)
]

df = spark.createDataFrame(data, ["date", "sales"])

# Define a rolling window — past 3 rows including current
window_spec = Window.orderBy("date").rowsBetween(-2, 0)

# Calculate moving average
df_with_ma = df.withColumn("moving_avg", avg(col("sales")).over(window_spec))

df_with_ma.show()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

# Start Spark session
spark = SparkSession.builder.appName("SalesRanking").getOrCreate()

# Sample sales transaction data
data = [
    ("North", "CustA", 500),
    ("North", "CustA", 700),
    ("North", "CustB", 600),
    ("South", "CustA", 800),
    ("South", "CustB", 750),
    ("South", "CustB", 900)
]

df = spark.createDataFrame(data, ["region", "customer_id", "sales"])

# Define window spec to partition by region and customer, order by sales DESC
window_spec = Window.partitionBy("region", "customer_id").orderBy(col("sales").desc())

# Add rank column
ranked_df = df.withColumn("sales_rank", rank().over(window_spec))

# Show result
ranked_df.show()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count

# Start Spark session
spark = SparkSession.builder.appName("FilterSingleOccurrences").getOrCreate()

# Sample data
data = [
    ("CustA", 100),
    ("CustB", 200),
    ("CustA", 150),
    ("CustC", 300),
    ("CustD", 400)
]

df = spark.createDataFrame(data, ["customer_id", "sales"])

# Step 1: Identify values that appear more than once
duplicates = df.groupBy("customer_id") \
               .agg(count("*").alias("cnt")) \
               .filter(col("cnt") > 1) \
               .select("customer_id")
duplicates.show()
# Step 2: Filter original DataFrame using semi join
filtered_df = df.join(duplicates, on="customer_id", how="inner")

filtered_df.show()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, lower, col, count
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Create Spark session
spark = SparkSession.builder.appName("TopNWords").getOrCreate()

# Sample DataFrame with text
data = [("Hello world spark spark",), ("Spark is fast and powerful",), ("Hello from the other side",)]
df = spark.createDataFrame(data, ["text"])
df.display()
# Step 1: Normalize and split text into words
words_df = df.select(explode(split(lower(col("text")), " ")).alias("word"))
words_df.display()
# Step 2: Count frequency of each word
word_counts = words_df.groupBy("word").agg(count("*").alias("freq"))
word_counts.display()
# Step 3: Order and extract top N
top_n = word_counts.orderBy(col("freq").desc()).limit(5)  # Change 5 to your desired N

top_n.show()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, lower, col, count
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Create Spark session
spark = SparkSession.builder.appName("TopNWords").getOrCreate()

# Sample DataFrame with text
data = [("Hello world spark spark",), ("Spark is fast and powerful",), ("Hello from the other side",)]
df = spark.createDataFrame(data, ["text"])
df.display()

df_explode = df.withColumn("word",explode(split(col("text")," "))).drop("text")

df_count = df_explode.groupBy("word").agg(count("*").alias("freq")).orderBy(col("freq").desc()).limit(5)

df_count.display()

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number, col

# Define your window spec
window_spec = Window.partitionBy("user_id", "product_id").orderBy(col("event_time").desc())

# Add row number within each group
df_ranked = df.withColumn("row_num", row_number().over(window_spec))

# Filter to keep only latest record per group
df_deduped = df_ranked.filter(col("row_num") == 1).drop("row_num")

df_deduped.show()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import least, greatest

spark = SparkSession.builder.getOrCreate()

# Sample data
data = [("Mumbai","Pune"),
        ("Pune","Mumbai"),
        ("Delhi","Bangalore"),
        ("Bangalore","Delhi")]

df = spark.createDataFrame(data, ["source", "destination"])

# Step 1: build canonical columns
df2 = df.withColumn("src_key", least("source", "destination")) \
        .withColumn("dst_key", greatest("source", "destination"))
df2.display()
# Step 2: drop cross-duplicates
deduped = df2.dropDuplicates(["src_key", "dst_key"]) \
             .select("source", "destination")

deduped.show(truncate=False)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, to_date, row_number, count, min, max, date_sub
)
from pyspark.sql.window import Window

# Sample data
data = [
    (1, '2025-03-01'), (1, '2025-03-02'), (1, '2025-03-03'),
    (1, '2025-03-04'), (1, '2025-03-06'), (1, '2025-03-10'),
    (1, '2025-03-11'), (1, '2025-03-12'), (1, '2025-03-13'),
    (1, '2025-03-14'), (1, '2025-03-25'), (1, '2025-03-26'),
    (1, '2025-03-27'), (1, '2025-03-28'), (1, '2025-03-29'),
    (1, '2025-03-30'), (2, '2025-03-01'), (2, '2025-03-02'),
    (2, '2025-03-03'), (2, '2025-03-04'), (3, '2025-03-01'),
    (3, '2025-03-02'), (3, '2025-03-03'), (3, '2025-03-04'),
    (3, '2025-03-04'), (3, '2025-03-04'), (3, '2025-03-05')
]

spark = SparkSession.builder.getOrCreate()

# 1. Create DataFrame and normalize types
df = (spark.createDataFrame(data, ['user_id', 'login_date'])
        .withColumn('login_date', to_date('login_date'))
        .dropDuplicates(['user_id', 'login_date'])  # dedupe same-day logins
     )
df.display()
# 2. Assign a row number per user ordered by date
window = Window.partitionBy('user_id').orderBy('login_date')
df = df.withColumn('rn', row_number().over(window))
df.display()
# 3. Compute a DATE-based “group key” by shifting each date back by its row number
df = df.withColumn('grp', date_sub(col('login_date'), col('rn')))
df.display()
# 4. Aggregate per user+grp to get streaks
streaks = (
    df.groupBy('user_id', 'grp')
      .agg(
         count('*').alias('consecutive_days'),
         min('login_date').alias('start_date'),
         max('login_date').alias('end_date')
      )
      .filter(col('consecutive_days') >= 5)
      .select('user_id', 'start_date', 'end_date', 'consecutive_days')
)

streaks.show()

word count

In [0]:
from pyspark.sql.functions import explode, split, lower, col

# Sample input
text_data = ["Hello world", "Hello Spark", "Spark is fast and scalable"]

# Create a DataFrame from the list of strings
df = spark.createDataFrame([(line,) for line in text_data], ["line"])
df.display()
df_temp = df.select(explode(split(lower(col("line")), " ")).alias("word"))
df_temp.display()
# Split lines into words, explode the array to rows, normalize case, group & count
word_counts = df.select(explode(split(lower(col("line")), " ")).alias("word")) \
                .groupBy("word") \
                .count()

# Show the result
word_counts.display()